<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-2/blob/main/CS6910_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

# Importing libraries related to Image Preprocessing
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Importing libraries related to CNN Model building
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Activation,Dropout,BatchNormalization




In [ ]:
# !pip install wandb
# import wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Creating a CNN: number of filters, size of filters, activation function has to be 
# user specified
img_shape=(256,256,3)
def build_cnn(conv_act , dense_act,num_filters, conv_filter_size, pool_filter_size, batch_norm, dim_final, dropout):
  # conv_act         : "List" activation used for convolution layer
  # dense_act        : "String" acitvation used for densely connected layers
  # num_filters      : "List" number of activation filters for each layer
  # conv_filter_size : "List" kernel sizes for convultion layers
  # pool_filter_size : "List" kernel sizes for maxpooling layers
  # batch_norm       : "Boolean" set to True, if you are using batch normalization
  # dim_final        : "Integer" dimensionality of output space after 5 blocks of convultion, maxpooling blocks
  # dropout          : "float or double" specify the dropout % for regularization (in decimals)
  
  model=Sequential()
  # Adding the first conv,activ,maxpool layers
  model.add(Conv2D(filters=num_filters[0],
                   kernel_size=conv_filter_size[0],
                   input_shape=img_shape))
  
  # Add batch normalization layer, if the user specifies
  if batch_norm:
    model.add(BatchNormalization())
  
  model.add(Activation(conv_act[0]))
  model.add(MaxPool2D(pool_size=pool_filter_size[0],strides=(2,2)))

  # Adding the next 4 layer blocks
  for i in range(1,5):
    model.add(Conv2D(filters=num_filters[i],kernel_size=conv_filter_size[i]))
    if batch_norm:
      model.add(BatchNormalization())
    model.add(Activation(conv_act[i]))
    model.add(MaxPool2D(pool_size=pool_filter_size[i],strides=(2,2)))
  
  # Flattening the feature map to a column vector
  model.add(Flatten())
  model.add(Dense(units=dim_final,activation=dense_act))
  # Adding dropout regularization
  model.add(Dropout(dropout))
  # Adding the final Dense layer with dimensions equal to number of classes and softmax activation
  model.add(Dense(10,activation="softmax"))

  return model


In [ ]:
conv=["relu","relu","relu","relu","relu"]
den="relu"
filt=[32,64,128,256,512]
conv_size=[(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)]
pool_size = [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
model=build_cnn(conv,den,filt,conv_size,pool_size,True,512,0.8)

Input pipeline for image classification reference :
https://towardsdatascience.com/what-is-the-best-input-pipeline-to-train-image-classification-models-with-tf-keras-eb3fe26d3cc5

In [ ]:
# Dataset without augmentation

# Enter the path of dataset
path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

# Training dataset:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    directory=path,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(256,256),
    shuffle=True,
    seed=19,
    validation_split=0.1,
    subset='training'
)
# Validation dataset:
valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    directory=path,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(256,256),
    shuffle=True,
    seed=19,
    validation_split=0.1,
    subset='validation'
)

Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.


In [ ]:
len_train,len_valid=len(train_dataset),len(valid_dataset)
# Retaining 10% of data for hyperparameter tuning
train_data=train_dataset.take(int(0.1*len_train))
valid_data=valid_dataset.take(int(0.1*len_valid))

In [ ]:
# Dataset with augmentation

training_data_augmentation=ImageDataGenerator(
    rescale=None,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.2,1.5],
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    data_format='channels_last',
    validation_split=0.1
)

# Validation data is not being augmented
validation_data_augmentation=ImageDataGenerator(
    validation_split=0.1
)

train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')



In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(name='loss'),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')])

history=model.fit(x=train_data,
                  epochs=50,
                  validation_data=valid_data,callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5)]
                  )

In [ ]:
def cnn_wandb():
  

In [ ]:
act_conv=["relu","relu","relu","relu","relu"]
act_den="relu"
num_filt=[32,64,128,256,512]
conv_size=[(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)]
pool_size = [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
model=build_cnn(conv,den,filt,conv_size,pool_size,True,512,0.8)

In [ ]:
# hyper parameters: activation function for convolution layer and dense layer,
# batch normalization, % of dropout
hyperparameters={
    
    'batch_norm': {
        'values':[True,False]
    },
    'num_filter':{
        'values':[32,64,128,256,512]

    },
    
   


}